# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайта) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [ ]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!pip install tokenizers matplotlib sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=82a88d2dc896c3db7866feee033640044afe38bbc5db5f895a4f4ad67180a010
  Stored in directory: /root/.cache/pip/wheels/d5/b2/a9/590d15767d34955f20a9a033e8db973b79cb5672d95790c0a9
Successfully built sklearn


In [ ]:
!pip install --upgrade jedi==0.17.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: parso
    Found existing installation: parso 0.8.3
    Uninstalling parso-0.8.3:
      Successfully uninstalled parso-0.8.3


In [ ]:
!pip install tokenizers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2023-04-20 06:19:15--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  20.8MB/s    in 6.4s    

2023-04-20 06:19:22 (18.1 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2023-04-20 06:19:22--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  18.3MB/s    in 4.1s    

2023-04-20 06:19:26 (15.9 MB/s) - ‘op

In [ ]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

In [ ]:
# убеждаемся, что всё работает
en_sents[-100], ru_sents[-100]

('enough for a new life in london?', 'достаточно для новой жизни в лондоне?')

In [ ]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [ ]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [ ]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [ ]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [ ]:
def encode(text, tokenizer, target=False):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]

In [ ]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, target=True) for t in ru_sents]

In [ ]:
# т.к. я оставила значения max_len_en/ru такими, какие они есть (не вводила сама),
# понадобилось перевести их значения из numpy.float64 в integer для корректной работы кода

max_len_en = int(np.mean([len(x) for x in X_en]))
max_len_ru = int(np.mean([len(x) for x in X_ru]))

In [ ]:
max_len_en, max_len_ru

(17, 16)

In [ ]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
print(PAD_IDX)
print(tokenizer_en.token_to_id('[PAD]'))

3
3


In [ ]:
# ещё в X_ru_out убрала -1 в maxlen, чтобы у X_en.shape и X_ru_out.shape разница тоже была на 1,
# как и в семинарской тетрадке

X_en = tf.keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=int(max_len_en), padding='post', value=tokenizer_en.token_to_id('[PAD]'))

X_ru_out = tf.keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru, padding='post', 
              value=tokenizer_en.token_to_id('[PAD]'))

X_ru_dec = tf.keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru, 
              padding='post', value=tokenizer_ru.token_to_id('[PAD]'))

In [ ]:
# миллион примеров 
X_en.shape, X_ru_out.shape

((1000000, 17), (1000000, 16))

In [ ]:
X_en_train, X_en_valid, X_ru_dec_train, X_ru_dec_valid, X_ru_out_train, X_ru_out_valid = train_test_split(X_en, 
                                                                                                      X_ru_dec, 
                                                                                                      X_ru_out, 
                                                                                                      test_size=0.05)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [ ]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
tf.keras.backend.clear_session()

# small model
#NUM_LAYERS = 2
#D_MODEL = 256
#NUM_HEADS = 8
#UNITS = 512
#DROPOUT = 0.1

# параметры с huggingface
NUM_LAYERS = 2
D_MODEL = 128
NUM_HEADS = 8
UNITS = 128
DROPOUT = 0.1


mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_en, max_len_ru])

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                                monitor='val_loss',
                                                verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

In [ ]:
# помимо текста в модель еще нужно передать целевую последовательность
# но не полную а без 1 последнего элемента
# а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
# мы сравниваем выход из модели с целевой последовательностью уже с этим последним элементом

# сдвинутые последовательности создаются выше
# X_ru_dec - это переводной текст без последнего элемента
# X_ru_out - это переводной текст с последним элементом

model.fit((X_en_train, X_ru_dec_train), X_ru_out_train, 
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=200,
             epochs=100,
             callbacks=[checkpoint]
             )

Epoch 1/100
2715/2715 [==============================] - ETA: 0s - loss: 2.9675 - accuracy: 0.2343
Epoch 1: val_loss improved from inf to 2.25724, saving model to model_ruen
2715/2715 [==============================] - 534s 189ms/step - loss: 2.9675 - accuracy: 0.2343 - val_loss: 2.2572 - val_accuracy: 0.2974
Epoch 2/100
2715/2715 [==============================] - ETA: 0s - loss: 2.1450 - accuracy: 0.3060
Epoch 2: val_loss improved from 2.25724 to 1.97648, saving model to model_ruen
2715/2715 [==============================] - 433s 160ms/step - loss: 2.1450 - accuracy: 0.3060 - val_loss: 1.9765 - val_accuracy: 0.3258
Epoch 3/100
2715/2715 [==============================] - ETA: 0s - loss: 1.9474 - accuracy: 0.3260
Epoch 3: val_loss improved from 1.97648 to 1.88432, saving model to model_ruen
2715/2715 [==============================] - 426s 157ms/step - loss: 1.9474 - accuracy: 0.3260 - val_loss: 1.8843 - val_accuracy: 0.3372
Epoch 4/100
2715/2715 [==============================] - ET

In [ ]:
def translate(text):
    input_ids = encode(text.lower(), tokenizer_en)

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post')

    
    
    output_ids = [tokenizer_ru.token_to_id('[CLS]') ]
    
    pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)
 
    
    while pred.numpy().argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[SEP]'), 
                                                            ]:
        if len(output_ids) > max_len_ru:
            break
        output_ids.append(pred.numpy().argmax(2)[0][-1])
        pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)

    return tokenizer_ru.decode(output_ids[1:])

In [ ]:
# проверяем, всё ли работает
translate("can you translate this sentence?")

In [ ]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [ ]:
translations = []

for i in range(len(en_sents_test)):
  translations.append(translate(en_sents_test[i]))

In [ ]:
bleus = []

for i, t in enumerate(translations):
  reference = tokenizer_ru.encode(t).tokens
  hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

In [ ]:
(sum(bleus)/len(bleus))*100


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

*Честно скажу, что прочитала всю главу и не нашла там информации по этому вопросу, поэтому обратилась к интернету и ниже пересказываю прочитанное там....*

Back translation (или "обратный перевод") - метод улучшения качества машинного перевода путём создания дополнительных синтетических данных. Например, у нас есть параллельный корпус данных en-ru и мы хотим обучить модель переводить предложения с английского на русский. Применяя метод обратного перевода, мы сначала обучаем модель переводить с русского на английский, используя наши параллельные данные. Когда мы это сделали, мы можем найти корпус текстов только для русского и перевести его с помощью нашей модели на английский, тем самым получив дополнительный синтетический параллельный корпус данных. Новые синтетические данные можно смешать с уже имевшимися и обучить модель снова. Качество перевода обещает значительно улучшиться!